# XGBoost

In [1]:
import xgboost as xgb
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

In [3]:
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))

In [4]:
calcular_auc = '"{}"'.format(os.path.join(current_folder, '..', 'Calcular AUC.ipynb'))

In [5]:
set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))

In [6]:
predicciones_csv = os.path.join(current_folder, '..', 'predictions.csv')

Cargo el df con los features.

In [7]:
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [8]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

Cargo los sets de entrenamiento, testing y predicción.

In [9]:
%run $set_de_entrenamiento_testing_y_prediccion

Entreno.

In [10]:
training = labels_training.merge(df_features, how='inner', on='person')

In [11]:
training.head()

,label,screen_resolution_height,screen_resolution_width,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,event_count,Cant visitas con Computadoras,Cant visitas con smartphone,vive en Brasil,compras Enero,compras Febrero,compras Marzo,compras Abril,compras Mayo
person,,,,,,,,,,,,,,,,,,,,,,,
0566e9c1,0,568.000000,320.000000,6.0,3.0,1.0,1.0,15.0,0.0,1.0,0.0,1.0,23.0,17.0,68,0.0,17.0,True,0,0,0,0,1
6ec7ee77,0,640.000000,360.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,1.0,True,0,0,0,0,0
abe7a2fb,0,640.000000,360.000000,9.0,14.0,1.0,0.0,9.0,0.0,4.0,6.0,0.0,31.0,22.0,96,0.0,22.0,True,0,0,0,0,0
34728364,0,640.000000,360.000000,0.0,5.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,24.0,4.0,37,0.0,4.0,True,0,0,0,0,0
db2c4d27,1,640.491228,429.140351,3.0,31.0,3.0,1.0,166.0,0.0,92.0,13.0,20.0,121.0,114.0,564,7.0,107.0,True,0,0,1,0,0


In [12]:
columns = list(training.columns)
columns.remove('label')

In [13]:
dtrain = xgb.DMatrix(training.loc[:, columns], label=training['label'])

Hiperparámetros:

In [14]:
param = {'max_depth': 4, 'eta': 0.54, 'gamma': 7, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

In [15]:
num_round = 10
bst = xgb.train(param, dtrain, num_round)

Predigo:

In [16]:
labels_test = labels_test.merge(df_features, how='inner', on='person')

In [17]:
test_matrix = xgb.DMatrix(labels_test.loc[:, columns])

In [18]:
labels_test['label_predicted'] = bst.predict(test_matrix)

Métrica de la predicción:

In [19]:
%run $calcular_auc

auc score: 0.8461204336204335


## Búsqueda de hiperparámetros

In [20]:
param = {
    'max_depth': 0,
    'eta': 0,
    'gamma': 0,
    'silent': 1,
    'objective': 'binary:logistic',
    'nthread': 10,
    'eval_metric': 'auc'
}

In [ ]:
results = pd.DataFrame(columns=['max_depth', 'eta', 'gamma', 'auc'])
index = 0
for max_depth, eta, gamma in np.ndindex((10, 10, 100)):
    param['max_depth'] = max_depth
    param['eta'] = eta/10
    param['gamma'] = gamma
    num_round = 10
    bst = xgb.train(param, dtrain, num_round)
    test_matrix = xgb.DMatrix(labels_test.loc[:, columns])
    labels_test['label_predicted'] = bst.predict(test_matrix)
    results.loc[index] = max_depth, eta/10, gamma, calculate_auc()
    index+=1

In [ ]:
results.head()

In [ ]:
results.loc[results['auc'].idxmax()]

## Predecir las labels desconocidas

In [ ]:
training = labels.merge(df_features, how='inner', on='person')

In [ ]:
columns = list(training.columns)
columns.remove('label')

In [ ]:
dtrain = xgb.DMatrix(training.loc[:, columns], label=training['label'])

In [ ]:
param['max_depth'] = 4
param['eta'] = 0.4
param['gamma'] = 0

In [ ]:
num_round = 10
bst = xgb.train(param, dtrain, num_round)

Predigo:

In [ ]:
labels_to_predict_with_features = labels_to_predict.merge(df_features, how='inner', on='person')

In [ ]:
assert(labels_to_predict.shape[0] == labels_to_predict_with_features.shape[0])

In [ ]:
matrix = xgb.DMatrix(labels_to_predict_with_features.loc[:, columns])

In [ ]:
labels_to_predict['label'] = bst.predict(matrix)

In [ ]:
# labels_to_predict.to_csv(predicciones_csv)